# 간단한 이미지 분류기

이 노트북은 사전 학습된 신경망을 사용하여 이미지를 분류하는 방법을 보여줍니다.

**배울 내용:**
- 사전 학습된 모델을 로드하고 사용하는 방법
- 이미지 전처리
- 이미지에 대한 예측 수행
- 신뢰도 점수 이해하기

**사용 사례:** 이미지에서 객체 식별 (예: "고양이", "강아지", "자동차" 등)

---


## 1단계: 필요한 라이브러리 가져오기

필요한 도구를 가져와 봅시다. 아직 모든 것을 이해하지 못해도 걱정하지 마세요!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## 2단계: 사전 학습된 모델 로드하기

우리는 **MobileNetV2**를 사용할 것입니다. 이 신경망은 이미 수백만 개의 이미지로 학습되었습니다.

이것을 **전이 학습**이라고 합니다 - 다른 사람이 학습시킨 모델을 사용하는 것이죠!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## 3단계: 헬퍼 함수

모델에 사용할 이미지를 로드하고 준비하는 함수를 만들어봅시다.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## 4단계: 샘플 이미지 테스트

인터넷에서 가져온 몇 가지 이미지를 분류해 봅시다!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### 각 이미지를 분류하세요


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## 5단계: 직접 이미지를 시도해보세요!

아래 URL을 원하는 이미지 URL로 교체하여 분류를 시도해보세요.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 방금 무슨 일이 있었나요?

1. **사전 학습된 모델을 로드했습니다** - MobileNetV2는 수백만 개의 이미지로 학습되었습니다.
2. **이미지를 전처리했습니다** - 모델에 맞게 크기를 조정하고 형식을 변환했습니다.
3. **모델이 예측을 수행했습니다** - 1000개의 객체 클래스에 대한 확률을 출력했습니다.
4. **결과를 디코딩했습니다** - 숫자를 사람이 읽을 수 있는 레이블로 변환했습니다.

### 신뢰 점수 이해하기

- **90-100%**: 매우 높은 신뢰도 (거의 확실히 정확함)
- **70-90%**: 높은 신뢰도 (아마도 정확함)
- **50-70%**: 어느 정도 신뢰도 있음 (정확할 가능성 있음)
- **50% 미만**: 신뢰도 낮음 (불확실함)

### 왜 예측이 틀릴 수 있을까요?

- **특이한 각도나 조명** - 모델은 일반적인 사진으로 학습되었습니다.
- **여러 객체가 있는 경우** - 모델은 주요 객체 하나를 예상합니다.
- **드문 객체** - 모델은 1000개의 카테고리만 알고 있습니다.
- **저화질 이미지** - 흐릿하거나 픽셀화된 이미지는 더 어렵습니다.

---


## 🚀 다음 단계

1. **다양한 이미지 시도하기:**
   - [Unsplash](https://unsplash.com)에서 이미지를 찾아보세요.
   - 마우스 오른쪽 클릭 → "이미지 주소 복사"로 URL 가져오기

2. **실험해보기:**
   - 추상 예술 작품에서는 어떤 결과가 나올까요?
   - 다양한 각도에서 객체를 인식할 수 있을까요?
   - 여러 객체를 처리하는 방식은 어떨까요?

3. **더 알아보기:**
   - [컴퓨터 비전 강의](../lessons/4-ComputerVision/README.md)를 탐색해보세요.
   - 직접 이미지 분류기를 학습시키는 방법을 배워보세요.
   - CNN(합성곱 신경망)이 어떻게 작동하는지 이해해보세요.

---

## 🎉 축하합니다!

최첨단 신경망을 사용하여 이미지 분류기를 직접 구축하셨습니다!

이 기술은 다음과 같은 분야에서 활용됩니다:
- Google Photos (사진 정리)
- 자율주행 자동차 (객체 인식)
- 의료 진단 (X-ray 분석)
- 품질 관리 (결함 감지)

계속 탐구하고 배우세요! 🚀



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전이 권위 있는 출처로 간주되어야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
